# INSTRUÇÃO PRÁTICA - IP-P008
## Revisao e consolidaçao do conteúdo de __Pandas__ avançado.
Dados de estações metereologicas de Salvador dos anos de 2003, 2013 e 2023, diponiveis em: [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). 

### Exercicio 1: Tratando series temporais por tempos
1. Carregando dados utilizando `DatetimeIndex`

In [32]:
import pandas as pd
import numpy as np

estacao_ssa_2003 = pd.read_csv('datasets/tratados/estacao_Salvador_2003.pkl', sep=",", encoding='latin-1')
estacao_ssa_2013 = pd.read_csv('datasets/tratados/estacao_Salvador_2013.pkl', sep=",", encoding='utf-8')
estacao_ssa_2023 = pd.read_csv('datasets/tratados/estacao_Salvador_2023.pkl', sep=",", encoding='latin-1')

In [33]:
# Cria o DateTimeIndex a partir da coluna DATA (YYYY-MM-DD) e HORA (UTC)
estacao_ssa_2003_index = pd.to_datetime(estacao_ssa_2003['DATA (YYYY-MM-DD)'] + ' ' + estacao_ssa_2003['HORA (UTC)'])
estacao_ssa_2013_index = pd.to_datetime(estacao_ssa_2013['DATA (YYYY-MM-DD)'] + ' ' + estacao_ssa_2013['HORA (UTC)'])
estacao_ssa_2023_index = pd.to_datetime(estacao_ssa_2023['Data'] + ' ' + estacao_ssa_2023['Hora UTC'])

# Remova as colunas originais DATA (YYYY-MM-DD) e HORA (UTC)
estacao_ssa_2003.drop(['DATA (YYYY-MM-DD)', 'HORA (UTC)'], axis=1, inplace=True)
estacao_ssa_2013.drop(['DATA (YYYY-MM-DD)', 'HORA (UTC)'], axis=1, inplace=True)
estacao_ssa_2023.drop(['Data', 'Hora UTC'], axis=1, inplace=True)

# Define a coluna DATA HORA como índice
estacao_ssa_2003.index = estacao_ssa_2003_index
estacao_ssa_2013.index = estacao_ssa_2013_index
estacao_ssa_2023.index = estacao_ssa_2023_index

2. Juntando os dois datasets em um só

In [35]:
estacao_ssa = pd.concat([estacao_ssa_2003, estacao_ssa_2013, estacao_ssa_2023])

3. Analisando os dataframes

In [36]:
estacao_ssa.replace(['-9999', -9999, -9999.0, '-9999.0'], value= np.nan, inplace=True)

In [38]:
# tratando precipitacao
print(estacao_ssa['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].unique())

# substiduindo valores , por .
estacao_ssa.loc[:, ('PRECIPITAÇÃO TOTAL, HORÁRIO (mm)')].replace(',', '.', regex=True, inplace=True)

# convertendo para float
estacao_ssa['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] = estacao_ssa['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].astype('float64')

# preenchendo valores NaN com valor anteriores
estacao_ssa.loc[:, ('PRECIPITAÇÃO TOTAL, HORÁRIO (mm)')].ffill(inplace=True)

[nan '0' ',2' '6' ',4' '4,8' '6,4' '1,4' '2,4' '1' ',6' '3' '21,8' '2,2'
 '6,8' ',8' '2,6' '4,6' '3,4' '8,4' '1,2' '1,8' '15,8' '12,4' '13,8' '4'
 '10,2' '2' '7,6' '1,6' '3,6' '3,2' '3,8' '4,2' '5,2' '5,8' '10,6' '6,2'
 '5,6' '9,6' '18,4' '7,4' '18' '16,2' '2,8' '5' '11,6' '10,4' '14,4' '52'
 '19' '24,4' '13' '21,6' '12,8' '17,6' '18,2' '11,8' '9' '9,4' '8,8' '22'
 '7,8' '8' '10' '11,2' '5,4' '12,2' '14,6' '7' '4,4' '12' '6,6' '15,2'
 '9,2' '13,2' '29,4' '7,2' '13,6' '0,8' '0,2' '0,4' '0,6' '9,8' '32,4'
 '20,6' '8,6' '16,6' '23,8' '14,2' '32,8' '12,6' '13,4' '11' '17' '15,6'
 '20,4' '17,2' '44,4' '16' '25' '16,8' '14' '8,2' '31,6' '24,2' '22,4'
 '16,4' '27,6' '38,4']


In [ ]:
# tratando temperaturas
print(np.unique(estacao_ssa['temperatura'].values.astype('str')))
colunas = estacao_ssa['temperatura'].columns.values.tolist()

# substiduindo valores , por .
for coluna in colunas:
    estacao_ssa.loc[:, ('temperatura', coluna)].replace(',', '.', regex=True, inplace=True)

# convertendo para float
estacao_ssa['temperatura'] = estacao_ssa['temperatura'].astype('float64')

# preenchendo valores NaN com valor anteriores
for coluna in colunas:
    estacao_ssa.loc[:, ('temperatura', coluna)].ffill(inplace=True)

### Exercicio 2: Utilizando eval() e query()
1. Comparação de desempenho